## Metadata

This notebook discusses how py4DSTEM handles metadata, including:
1. How metadata is stored and retrieved in-program
2. How metadata is edited, and how to tell py4DSTEM to permanently scrape new metadata from input files

In [1]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import py4DSTEM

In [2]:
# Load data

fp = "/home/ben/Data/20180830_zeltman_smallsampledatasets/1_10x10 ss=100 alpha=p48 spot 11 cl=1200 300kV RT bin=4 0p5sec.dm3"
dc = py4DSTEM.readwrite.read(fp)
dc.set_scan_shape(10,10)

/home/ben/Data/20180830_zeltman_smallsampledatasets/1_10x10 ss=100 alpha=p48 spot 11 cl=1200 300kV RT bin=4 0p5sec.dm3 is not a py4DSTEM file.  Reading with hyperspy...


#### How metadata is stored and retreived in-program

In [16]:
# Metadata lives in a py4DSTEM Metadata object.
# Loading new datacube from a file creates a DataCube instance and Matadata instance
# The datacube contains a pionter to the metadata at datacube.metadata

metadata = dc.metadata
type(metadata)

py4DSTEM.process.datastructure.metadata.Metadata

In [4]:
# The Metadata instance's metadata lives in metdata.metadata.  How meta! ;)
# It's structure is simple: metadata.metadata contains a handful of dictionaries, whose key value pairs constitute
# the metadata
# For manually searching metadata, just examine these directly

dc.metadata.metadata.__dict__.keys()

dict_keys(['__name__', 'microscope', 'sample', 'user', 'calibration', 'comments'])

In [5]:
dc.metadata.metadata.microscope

{'accelerating_voltage': 300.0,
 'accelerating_voltage_units': '',
 'camera_length': 195.0,
 'camera_length_units': '',
 'C2_aperture': '',
 'convergence_semiangle_mrad': '',
 'spot_size': '',
 'scan_rotation_degrees': '',
 'dwell_time': '',
 'dwell_time_units': '',
 'scan_size_Ny': '',
 'scan_size_Nx': '',
 'R_pix_size': '',
 'R_pix_units': '',
 'K_pix_size': '',
 'K_pix_units': '',
 'probe_FWHM_nm': '',
 'acquisition_date': '',
 'original_filename': '1_10x10 ss=100 alpha=p48 spot 11 cl=1200 300kV RT bin=4 0p5sec.dm3'}

In [6]:
dc.metadata.metadata.microscope['accelerating_voltage']

300.0

In [7]:
# If you already known the key of the field you need, you can get it's value in general with

dc.metadata.get_metadata_item('accelerating_voltage')

300.0

In [8]:
# The metadata in these dictionaries constitues everything py4DSTEM knows to scrape from the metadata hyperspy
# finds and places in its DictionaryTreeBrowser objects.  These objects are also stored in py4DSTEM's Metadata
# objects, so they can always be retreived and scoured manually if needs-be.  They live in
# metadata.original_metadata.

dc.metadata.original_metadata

In [13]:
dc.metadata.original_metadata.__dict__.keys()

dict_keys(['__name__', 'shortlist', 'all'])

In [14]:
type(dc.metadata.original_metadata.shortlist)

hyperspy.misc.utils.DictionaryTreeBrowser

In [15]:
dc.metadata.original_metadata.shortlist

├── Acquisition_instrument
│   └── TEM
│       ├── acquisition_mode = TEM
│       ├── beam_current = 0.005
│       ├── beam_energy = 300.0
│       ├── camera_length = 195.0
│       ├── magnification = 1200.0
│       └── microscope = FEI Tecnai Remote
├── General
│   ├── date = 2018-06-11
│   ├── original_filename = 1_10x10 ss=100 alpha=p48 spot 11 cl=1200 300kV RT bin=4 0p5sec.dm3
│   ├── time = 16:22:54
│   └── title = 1_10x10 ss=100 alpha=p48 spot 11 cl=1200 300kV RT bin=4 0p5sec
└── Signal
    ├── Noise_properties
    │   └── Variance_linear_model
    │       ├── gain_factor = 1.0
    │       └── gain_offset = 0.0
    ├── binned = False
    ├── quantity = Intensity
    └── signal_type = 

In [18]:
# You can search and retrieve items from these hyperspy objects with

dc.metadata.search_hs_tree('beam_current', dc.metadata.original_metadata.shortlist)

(True, 0.005)

#### Editing and scraping new metadata

In [23]:
# To add new metadata or edit existing metadata, you can either use the method

dc.metadata.add_metadata_item('sample','no moon; a space station',dc.metadata.metadata.sample)

In [24]:
dc.metadata.get_metadata_item('sample')

'no moon; a space station'

In [27]:
# Or just use the dictionaries themselves

comment = "Don't be too proud of this technological terror you've constructed. " + \
          "The ability to destroy a planet is insignificant next to the powers of the force."

dc.metadata.metadata.comments["DVs cautionary"] = comment

In [29]:
dc.metadata.get_metadata_item('DVs cautionary')

"Don't be too proud of this technological terror you've constructed. The ability to destroy a planet is insignificant next to the powers of the force."

In [30]:
# To tell py4DSTEM to automatically scrape an item of metadata from the DictionaryTreeBrowsers on load, edit
# the Metadata method setup_search_dicts(), in ./process/datastructure/metadata.py, with key:value pairs specifying the py4DSTEM:hyperspy strings associated
# with the metadata item of interest

dc.metadata.setup_metadata_search_dicts?